^C
Note: you may need to restart the kernel to use updated packages.


In [5]:
import tensorflow as tf
import numpy as np
import cv2

import dlib
import pickle

In [6]:
def get_model():
    backbone = tf.keras.applications.EfficientNetB2(
        input_shape=(96, 96, 3),
        include_top=False,
        weights=None
    )
    model = tf.keras.Sequential([
        backbone,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(7, activation='softmax')
    ])
    return model

In [7]:
model = get_model()
model.load_weights("best_weights.h5") # Load the saved weights 

In [8]:
# Load LabelEncoder 
def load_object(name):
    pickle_obj = open(f"{name}.pck","rb")
    obj = pickle.load(pickle_obj)
    return obj

Le = load_object("LabelEncoder")

c:\Users\345d0\anaconda3\Lib\site-packages\sklearn\base.py:347: InconsistentVersionWarning: Trying to unpickle estimator LabelEncoder from version 0.22.2.post1 when using version 1.3.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [9]:
def ProcessImage(image):
    image = tf.convert_to_tensor(image)
    image = tf.image.resize(image , [96 , 96] , method="bilinear")
    image = tf.expand_dims(image , 0)
    return image

def RealtimePrediction(image , model, encoder_):
    prediction = model.predict(image)
    prediction = np.argmax(prediction , axis = 1)
    return encoder_.inverse_transform(prediction)[0]

def rect_to_bb(rect):
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y
    return (x, y, w, h)

In [10]:
import cv2
import dlib

# Function to convert dlib rect to bounding box coordinates
def rect_to_bb(rect):
    x = rect.left()
    y = rect.top()
    w = rect.right() - x
    h = rect.bottom() - y
    return (x, y, w, h)

# Initialize video capture
VideoCapture = cv2.VideoCapture(0)

# Load dlib face detector
detector = dlib.get_frontal_face_detector()

while True:
    ret, frame = VideoCapture.read()
    
    if not ret:
        break
    
    # Convert frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the grayscale image
    rects = detector(gray, 0)

    if len(rects) >= 1:
        for rect in rects:
            (x, y, w, h) = rect_to_bb(rect)
            
            # Crop the region of interest around the face
            img = gray[y-10: y+h+10, x-10: x+w+10]
            
            # Skip processing if the cropped image has invalid dimensions
            if img.shape[0] == 0 or img.shape[1] == 0:
                cv2.imshow("Frame", frame)
            else:
                # Convert to RGB and process the image
                img = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
                img = ProcessImage(img)
                
                # Make predictions in real-time
                out = RealtimePrediction(img, model, Le)
                
                # Draw bounding box and prediction text
                cv2.rectangle(frame, (x, y), (x+w, y+h), (0, 255, 0), 2)
                z = y - 15 if y - 15 > 15 else y + 15
                cv2.putText(frame, str(out), (x, z), cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0, 255, 0), 2)
                
        # Display the frame with detected faces and predictions
        cv2.imshow("Frame", frame)
    else:
        # Show the frame even if no faces are detected
        cv2.imshow("Frame", frame)

    # Press 'q' to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the capture and close windows
VideoCapture.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 151ms/step


KeyboardInterrupt: 

: 